## Imports

In [ ]:
import datetime
import gc

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [ ]:
cuda_use_gpus(1)

In [ ]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Config

In [ ]:
feature_list_id = 'oofp_bradleypallen_mlp_activations'

In [ ]:
NUM_FOLDS = 5
NUM_EPOCHS = 200
BATCH_SIZE = 512

In [ ]:
RANDOM_SEED = 42

In [ ]:
np.random.seed(RANDOM_SEED)

## Read Data

In [ ]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [ ]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [ ]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [ ]:
y_train = load(features_data_folder + 'y_train.pickle')

In [ ]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [ ]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

## Train Models & Compute Out-of-Fold Predictions

In [ ]:
model_params = {
    'num_dense_1': 250,
    'num_dense_2': 120,
    'num_dense_3': 250,
    'num_dense_4': 120,
    'dense_dropout_rate': 0.5,
}

In [ ]:
def create_model_question_branch():
    input_q = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    
    embedding_q = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )(input_q)

    timedist_q = TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu',
    ))(embedding_q)

    lambda_q = Lambda(
        lambda x: K.max(x, axis=1),
        output_shape=(EMBEDDING_DIM, )
    )(timedist_q)
    
    output_q = lambda_q
    return input_q, output_q

In [ ]:
def zero_loss(y_true, y_pred):
    return K.zeros((1,))

In [ ]:
def create_dense_block(input_layer, size, dropout_rate, name=None):
    dense = Dense(size)(input_layer)
    bn = BatchNormalization()(dense)
    relu = Activation('relu')(bn)
    dropout = Dropout(dropout_rate, name=name)(relu)
    return dropout

In [ ]:
def create_model(params):
    q1_input, q1_output = create_model_question_branch()
    q2_input, q2_output = create_model_question_branch()
    
    merged_outputs = Concatenate()([q1_output, q2_output])

    dense_1 = create_dense_block(merged_outputs, params['num_dense_1'], params['dense_dropout_rate'], name='feature_output')
    dense_2 = create_dense_block(dense_1, params['num_dense_2'], params['dense_dropout_rate'])
    dense_3 = create_dense_block(dense_2, params['num_dense_3'], params['dense_dropout_rate'])
    dense_4 = create_dense_block(dense_3, params['num_dense_4'], params['dense_dropout_rate'])

    bn_final = BatchNormalization()(dense_4)
    output = Dense(1, activation='sigmoid', name='target_output')(bn_final)
    
    model = Model(
        inputs=[q1_input, q2_input],
        outputs=[output, dense_1],
    )
    
    model.compile(
        loss={'target_output': 'binary_crossentropy', 'feature_output': zero_loss},
        loss_weights={'target_output': 1.0, 'feature_output': 0.0},
        optimizer='nadam',
        metrics=None,
    )

    return model

In [ ]:
feature_output_size = model_params['num_dense_1']

In [ ]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [ ]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [ ]:
y_train_oofp = np.zeros_like(y_train, dtype='float32')
y_train_oofp_features = np.zeros((len(y_train), feature_output_size), dtype='float32')

In [ ]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS), dtype='float32')
y_test_oofp_features = np.zeros((len(X_test_q1), feature_output_size), dtype='float32')

In [ ]:
for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = X_train_q1[ix_train]
    X_fold_train_q2 = X_train_q2[ix_train]

    X_fold_val_q1 = X_train_q1[ix_val]
    X_fold_val_q2 = X_train_q2[ix_val]

    y_fold_train = y_train[ix_train]
    y_fold_val = y_train[ix_val]
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2],
        [y_fold_train, np.zeros((len(y_fold_train), feature_output_size))],
        
        validation_data=(
            [X_fold_val_q1, X_fold_val_q2],
            [y_fold_val, np.zeros((len(y_fold_val), feature_output_size))],
        ),

        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val], y_train_oofp_features[ix_val] = model.predict(
        [X_train_q1[ix_val], X_train_q2[ix_val]],
        batch_size=1024,
        verbose=1
    )
    
    if fold_num + 1 == NUM_FOLDS:
        y_test_oofp, y_test_oofp_features = model.predict(
            [X_test_q1, X_test_q2],
            batch_size=1024,
            verbose=1
        )
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_val_q1
    del X_fold_val_q2
    del model
    gc.collect()

In [ ]:
cv_score = log_loss(y_train, y_train_oofp)
print('CV score:', cv_score)

## Save feature names

In [ ]:
feature_names = [
    'oofp_bradleypallen_mlp_activations',
]

In [ ]:
save_feature_names(feature_names, feature_list_id)

## Save Train features

In [ ]:
save_feature_list(y_train_oofp_features, 'train', feature_list_id)

## Save Test features

In [ ]:
save_feature_list(y_test_oofp_features, 'test', feature_list_id)

## Explore

In [ ]:
pd.DataFrame(y_train_oofp_features).describe().T

In [ ]:
pd.DataFrame(y_test_oofp_features).describe().T